#### Importing libraries for processing of database

In [15]:
library(RSQLite)
library(dplyr)

In [16]:
#### Reading the sqlite database using src_sqlite function

In [17]:
my_db <- src_sqlite('database.sqlite')

#### Selecting required fields for player questions from player and player_attributes table. Taking sum of gk attributes for goalkeeper question. Grouping all the fields using for player_api_id

In [18]:
p <- tbl(my_db, sql("SELECT  Player.player_api_id, player_name, overall_rating, gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes as gk_attributes from Player LEFT JOIN Player_Attributes on Player.player_api_id = Player_Attributes.player_api_id group by Player.player_api_id"))

#### Selecting entire match table as it is required for both player and team files

In [19]:
mtable <- as.data.frame(tbl(my_db, sql("SELECT * from Match")))

#### Stripping the match table off home_player_(1-22) and away_player_(1-22) using column numbers (56-77), so that we get ids of players appearing in the matches. Grouping them using table to get appearances for each player_api_id and merging both to get playernames and apperances. Binding all these dataframes to make a common dataframe. Merging this data with our player data to create player database - pdb.

In [20]:
m <- mtable[56:77]
apps<-unlist(m)
apps<-as.data.frame(table(apps))
names(apps) <- c("player_api_id", "total_appearances")
pdb<-merge(x=apps,y=p,by=c("player_api_id"), all = TRUE)

#### Omitting NA values and selecting all home_player_(1-22) and away_player_(1-22) ids, each to a single variable alongwith league_id

In [21]:
r1 <- na.omit(select(mtable,player_api_id = home_player_1, league_id))
r2 <- na.omit(select(mtable,player_api_id = home_player_2, league_id))
r3 <- na.omit(select(mtable,player_api_id = home_player_3, league_id))
r4 <- na.omit(select(mtable,player_api_id = home_player_4, league_id))
r5 <- na.omit(select(mtable,player_api_id = home_player_5, league_id))
r6 <- na.omit(select(mtable,player_api_id = home_player_6, league_id))
r7 <- na.omit(select(mtable,player_api_id = home_player_7, league_id))
r8 <- na.omit(select(mtable,player_api_id = home_player_8, league_id))
r9 <- na.omit(select(mtable,player_api_id = home_player_9, league_id))
r10 <- na.omit(select(mtable,player_api_id = home_player_10, league_id))
r11 <- na.omit(select(mtable,player_api_id = home_player_11, league_id))


In [22]:
r12 <- na.omit(select(mtable,player_api_id = away_player_1, league_id))
r13 <- na.omit(select(mtable,player_api_id = away_player_2, league_id))
r14 <- na.omit(select(mtable,player_api_id = away_player_3, league_id))
r15 <- na.omit(select(mtable,player_api_id = away_player_4, league_id))
r16 <- na.omit(select(mtable,player_api_id = away_player_5, league_id))
r17 <- na.omit(select(mtable,player_api_id = away_player_6, league_id))
r18 <- na.omit(select(mtable,player_api_id = away_player_7, league_id))
r19 <- na.omit(select(mtable,player_api_id = away_player_8, league_id))
r20 <- na.omit(select(mtable,player_api_id = away_player_9, league_id))
r21 <- na.omit(select(mtable,player_api_id = away_player_10, league_id))
r22 <- na.omit(select(mtable,player_api_id = away_player_11, league_id))

In [23]:
rdf <- rbind(r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,r11,r12,r13,r14,r15,r16,r17,r18,r19,r20,r21,r22)


#### Using aggregate function to get unique league_id for each player_api_id and sum through the aggregate function to get no. of leagues a player has played in

In [24]:
rall <- aggregate(league_id ~ player_api_id, rdf, FUN = function(x) { length(unique(x))})

#### Changing dataframe column names to required field names

In [25]:
names(rall) <- c("player_api_id","leagues_played")

#### Merging this data with player database pdb and writing pdb into player.csv

In [26]:
pdb<-merge(x=rall,y=pdb,by=c("player_api_id"), all = TRUE)

In [27]:
write.csv(pdb, file = "player.csv",row.names=FALSE)

#### Selecting required fields for team questions from team and team_attributes table. Taking sum of attributes for attributes question. Grouping all the fields using for team_api_id

In [28]:
t <- tbl(my_db, sql("SELECT  Team.team_api_id, team_long_name, max(buildupplayspeed + buildupplaydribbling + buildupplaypassing) as buildup_ovr, max(chancecreationpassing + chancecreationcrossing + chancecreationshooting) as chance_ovr, max(defencepressure + defenceaggression + defenceteamwidth) as defence_ovr from Team LEFT JOIN Team_Attributes on Team.team_api_id = Team_Attributes.team_api_id group by Team.team_api_id"))

#### Stipping match table off home_team_id and home_team_goal (columns: 8,10) and away_team_id and away_team_goal (columns: 9,11). Grouping by group by function and in that summarising by getting sum of home and away goals and then binding them. And them merging this with our team data into team database tdb.

In [29]:
hgoals<-mtable[,c(8,10)]
hgoals<-hgoals%>%group_by(team_api_id=home_team_api_id)%>%summarise(total_goals=sum(home_team_goal))

agoals<-mtable[,c(9,11)]
agoals<-agoals%>%group_by(team_api_id=away_team_api_id)%>%summarise(total_goals=sum(away_team_goal))
goals<-rbind(hgoals,agoals)
goals<- goals%>%group_by(team_api_id)%>%summarise(total_goals = sum(total_goals))
tdb<-merge(x=goals,y=t,by=c("team_api_id"),all = TRUE)


#### Write team database tdb into team.csv

In [30]:
write.csv(tdb, file = "team.csv",row.names=FALSE)